# Etude de cas sur l'analyse des ressources humaines
Chaque année, à peu près 15% des employées quitte l'entreprise et doivent être remplacés par des talents disponible sur le marché du travail. Le but ici est de créer un modèle capable de prédire l'attrition d'un employé.

In [110]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score,f1_score,recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

## Analyse

### Lecture

In [111]:
def read_data(data):
    """Lire un dataframe et afficher la tête"""
    X = pd.read_csv(data)
    print('Number of rows : ', X.shape[0])
    print('Number of columns : ', X.shape[1])
    if 'Unnamed: 0' in X.columns:
        X.drop('Unnamed: 0', axis=1, inplace=True)

    return X

HR_CASE_STUDY = 'hr-analytics-case-study/'

In [112]:
employee_survey = read_data(HR_CASE_STUDY + 'employee_survey_data.csv')
employee_survey.head()

Number of rows :  4410
Number of columns :  4


,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance
0,1,3.0,4.0,2.0
1,2,3.0,2.0,4.0
2,3,2.0,2.0,1.0
3,4,4.0,4.0,3.0
4,5,4.0,1.0,3.0


In [113]:
manager_survey = read_data(HR_CASE_STUDY + 'manager_survey_data.csv')
manager_survey.head()

Number of rows :  4410
Number of columns :  3


,EmployeeID,JobInvolvement,PerformanceRating
0,1,3,3
1,2,2,4
2,3,3,3
3,4,2,3
4,5,3,3


In [114]:
in_time = read_data(HR_CASE_STUDY + 'in_time.csv')
in_time.head()

Number of rows :  4410
Number of columns :  262


,2015-01-01,2015-01-02,2015-01-05,2015-01-06,2015-01-07,2015-01-08,2015-01-09,2015-01-12,2015-01-13,2015-01-14,...,2015-12-18,2015-12-21,2015-12-22,2015-12-23,2015-12-24,2015-12-25,2015-12-28,2015-12-29,2015-12-30,2015-12-31
0,NaN,2015-01-02 09:43:45,2015-01-05 10:08:48,2015-01-06 09:54:26,2015-01-07 09:34:31,2015-01-08 09:51:09,2015-01-09 10:09:25,2015-01-12 09:42:53,2015-01-13 10:13:06,NaN,...,NaN,2015-12-21 09:55:29,2015-12-22 10:04:06,2015-12-23 10:14:27,2015-12-24 10:11:35,NaN,2015-12-28 10:13:41,2015-12-29 10:03:36,2015-12-30 09:54:12,2015-12-31 10:12:44
1,NaN,2015-01-02 10:15:44,2015-01-05 10:21:05,NaN,2015-01-07 09:45:17,2015-01-08 10:09:04,2015-01-09 09:43:26,2015-01-12 10:00:07,2015-01-13 10:43:29,NaN,...,2015-12-18 10:37:17,2015-12-21 09:49:02,2015-12-22 10:33:51,2015-12-23 10:12:10,NaN,NaN,2015-12-28 09:31:45,2015-12-29 09:55:49,2015-12-30 10:32:25,2015-12-31 09:27:20
2,NaN,2015-01-02 10:17:41,2015-01-05 09:50:50,2015-01-06 10:14:13,2015-01-07 09:47:27,2015-01-08 10:03:40,2015-01-09 10:05:49,2015-01-12 10:03:47,2015-01-13 10:21:26,NaN,...,2015-12-18 10:15:14,2015-12-21 10:10:28,2015-12-22 09:44:44,2015-12-23 10:15:54,2015-12-24 10:07:26,NaN,2015-12-28 09:42:05,2015-12-29 09:43:36,2015-12-30 09:34:05,2015-12-31 10:28:39
3,NaN,2015-01-02 10:05:06,2015-01-05 09:56:32,2015-01-06 10:11:07,2015-01-07 09:37:30,2015-01-08 10:02:08,2015-01-09 10:08:12,2015-01-12 10:13:42,2015-01-13 09:53:22,NaN,...,2015-12-18 10:17:38,2015-12-21 09:58:21,2015-12-22 10:04:25,2015-12-23 10:11:46,2015-12-24 09:43:15,NaN,2015-12-28 09:52:44,2015-12-29 09:33:16,2015-12-30 10:18:12,2015-12-31 10:01:15
4,NaN,2015-01-02 10:28:17,2015-01-05 09:49:58,2015-01-06 09:45:28,2015-01-07 09:49:37,2015-01-08 10:19:44,2015-01-09 10:00:50,2015-01-12 10:29:27,2015-01-13 09:59:32,NaN,...,2015-12-18 09:58:35,2015-12-21 10:03:41,2015-12-22 10:10:30,2015-12-23 10:13:36,2015-12-24 09:44:24,NaN,2015-12-28 10:05:15,2015-12-29 10:30:53,2015-12-30 09:18:21,2015-12-31 09:41:09


In [115]:
out_time = read_data(HR_CASE_STUDY + 'out_time.csv')
out_time.head()

Number of rows :  4410
Number of columns :  262


,2015-01-01,2015-01-02,2015-01-05,2015-01-06,2015-01-07,2015-01-08,2015-01-09,2015-01-12,2015-01-13,2015-01-14,...,2015-12-18,2015-12-21,2015-12-22,2015-12-23,2015-12-24,2015-12-25,2015-12-28,2015-12-29,2015-12-30,2015-12-31
0,NaN,2015-01-02 16:56:15,2015-01-05 17:20:11,2015-01-06 17:19:05,2015-01-07 16:34:55,2015-01-08 17:08:32,2015-01-09 17:38:29,2015-01-12 16:58:39,2015-01-13 18:02:58,NaN,...,NaN,2015-12-21 17:15:50,2015-12-22 17:27:51,2015-12-23 16:44:44,2015-12-24 17:47:22,NaN,2015-12-28 18:00:07,2015-12-29 17:22:30,2015-12-30 17:40:56,2015-12-31 17:17:33
1,NaN,2015-01-02 18:22:17,2015-01-05 17:48:22,NaN,2015-01-07 17:09:06,2015-01-08 17:34:04,2015-01-09 16:52:29,2015-01-12 17:36:48,2015-01-13 18:00:13,NaN,...,2015-12-18 18:31:28,2015-12-21 17:34:16,2015-12-22 18:16:35,2015-12-23 17:38:18,NaN,NaN,2015-12-28 17:08:38,2015-12-29 17:54:46,2015-12-30 18:31:35,2015-12-31 17:40:58
2,NaN,2015-01-02 16:59:14,2015-01-05 17:06:46,2015-01-06 16:38:32,2015-01-07 16:33:21,2015-01-08 17:24:22,2015-01-09 16:57:30,2015-01-12 17:28:54,2015-01-13 17:21:25,NaN,...,2015-12-18 17:02:23,2015-12-21 17:20:17,2015-12-22 16:32:50,2015-12-23 16:59:43,2015-12-24 16:58:25,NaN,2015-12-28 16:43:31,2015-12-29 17:09:56,2015-12-30 17:06:25,2015-12-31 17:15:50
3,NaN,2015-01-02 17:25:24,2015-01-05 17:14:03,2015-01-06 17:07:42,2015-01-07 16:32:40,2015-01-08 16:53:11,2015-01-09 17:19:47,2015-01-12 17:13:37,2015-01-13 17:11:45,NaN,...,2015-12-18 17:55:23,2015-12-21 16:49:09,2015-12-22 17:24:00,2015-12-23 17:36:35,2015-12-24 16:48:21,NaN,2015-12-28 17:19:34,2015-12-29 16:58:16,2015-12-30 17:40:11,2015-12-31 17:09:14
4,NaN,2015-01-02 18:31:37,2015-01-05 17:49:15,2015-01-06 17:26:25,2015-01-07 17:37:59,2015-01-08 17:59:28,2015-01-09 17:44:08,2015-01-12 18:51:21,2015-01-13 18:14:58,NaN,...,2015-12-18 17:52:48,2015-12-21 17:43:35,2015-12-22 18:07:57,2015-12-23 18:00:49,2015-12-24 17:59:22,NaN,2015-12-28 17:44:59,2015-12-29 18:47:00,2015-12-30 17:15:33,2015-12-31 17:42:14


In [116]:
general_data = read_data(HR_CASE_STUDY + 'general_data.csv')
general_data.head()

Number of rows :  4410
Number of columns :  24


,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,...,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,...,1.0,Y,11,8,0,1.0,6,1,0,0
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,2,Female,...,0.0,Y,23,8,1,6.0,3,5,1,4
2,32,No,Travel_Frequently,Research & Development,17,4,Other,1,3,Male,...,1.0,Y,15,8,3,5.0,2,5,0,3
3,38,No,Non-Travel,Research & Development,2,5,Life Sciences,1,4,Male,...,3.0,Y,11,8,3,13.0,5,8,7,5
4,32,No,Travel_Rarely,Research & Development,10,1,Medical,1,5,Male,...,4.0,Y,12,8,2,9.0,2,6,0,4


### Analyse de General data

In [117]:
gen_cat_vars = general_data.drop('Attrition', axis=1).select_dtypes(include=['object']).columns.tolist()
print('Liste des caractéristiques categoriques  : ', gen_cat_vars)
print('Nombre des caractéristiques categoriques : ', len(gen_cat_vars))

Liste des caractéristiques categoriques  :  ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'Over18']
Nombre des caractéristiques categoriques :  7


In [118]:
gen_num_vars = general_data.select_dtypes(exclude=['object']).columns.tolist()
print('Liste des caractéristiques numériques  : ', gen_num_vars)
print('Nombre des caractéristiques numériques : ', len(gen_num_vars))

Liste des caractéristiques numériques  :  ['Age', 'DistanceFromHome', 'Education', 'EmployeeCount', 'EmployeeID', 'JobLevel', 'MonthlyIncome', 'NumCompaniesWorked', 'PercentSalaryHike', 'StandardHours', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager']
Nombre des caractéristiques numériques :  16


In [119]:
gen_col_na = general_data.columns[general_data.isnull().sum() > 0].tolist()
print('Les colonnes ayant au moin une valeur vide : ', gen_col_na)
print('Nombre de valeur vide avec pour chaque colonne')
general_data[gen_col_na].isnull().sum()

Les colonnes ayant au moin une valeur vide :  ['NumCompaniesWorked', 'TotalWorkingYears']
Nombre de valeur vide avec pour chaque colonne


NumCompaniesWorked    19
TotalWorkingYears      9
dtype: int64

## Feature engineering

In [120]:
def label_travel(x):
    if x=="Travel_Rarely":
        return 1
    if x=="Travel_Frequently":
        return 2
    if x=="Non-Travel":
        return 0
    return 0

general_data['Attrition'] = [1 if a=="Yes" else 0 for a in general_data['Attrition']]
general_data['MonthlyIncome'] = np.log10(general_data['MonthlyIncome'].values)
general_data['BusinessTravel'] = list(map(label_travel,general_data['BusinessTravel'].values))
gen_cat_vars.remove('BusinessTravel')
gen_num_vars.remove('EmployeeID')
general_data = general_data.fillna(0)
general_data.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,...,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,51,0,1,Sales,6,2,Life Sciences,1,1,Female,...,1.0,Y,11,8,0,1.0,6,1,0,0
1,31,1,2,Research & Development,10,1,Life Sciences,1,2,Female,...,0.0,Y,23,8,1,6.0,3,5,1,4
2,32,0,2,Research & Development,17,4,Other,1,3,Male,...,1.0,Y,15,8,3,5.0,2,5,0,3
3,38,0,0,Research & Development,2,5,Life Sciences,1,4,Male,...,3.0,Y,11,8,3,13.0,5,8,7,5
4,32,0,1,Research & Development,10,1,Medical,1,5,Male,...,4.0,Y,12,8,2,9.0,2,6,0,4


## Créer le modèle

In [121]:
y = general_data['Attrition']
X = general_data.drop('Attrition', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=20)

### One hot encoding

In [122]:
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
X_enc = encoder.fit_transform(X_train[gen_cat_vars])
X_train_cat = pd.DataFrame(X_enc, columns=encoder.get_feature_names_out(), index=X_train.index)

X_train = pd.concat([X_train_cat, X_train[gen_num_vars]], axis=1)
X_train.head()

,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,Gender_Female,...,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
1130,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,4.793022,5.0,17,8,1,9.0,5,5,1,2
187,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,5.080302,3.0,11,8,0,37.0,5,10,7,7
3106,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,4.799754,3.0,12,8,1,32.0,3,1,0,0
2080,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,4.371253,1.0,16,8,0,6.0,4,6,3,3
1035,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,4.353724,4.0,18,8,0,7.0,2,3,1,2


In [123]:
X_test_enc = encoder.transform(X_test[gen_cat_vars])
X_test_cat = pd.DataFrame(X_test_enc, columns=encoder.get_feature_names_out(), index=X_test.index)

X_test = pd.concat([X_test_cat, X_test[gen_num_vars]], axis=1)
X_test.head()

,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,Gender_Female,...,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
913,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,4.560265,3.0,16,8,2,10.0,0,4,0,3
2163,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,4.442950,1.0,11,8,0,19.0,3,19,11,9
3819,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,4.986996,1.0,18,8,2,4.0,3,4,1,3
834,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,4.553883,4.0,20,8,1,8.0,1,5,0,3
1745,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,4.709779,5.0,18,8,0,20.0,2,18,1,12


In [124]:
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)
print("ACC:",accuracy_score(y_test, y_pred))
print("recall:",recall_score(y_test, y_pred))
print("F1:",f1_score(y_test, y_pred))

ACC: 0.9931972789115646
recall: 0.9602649006622517
F1: 0.9797297297297297
